In [ ]:
import tensorflow as tf

# 인코더 모델 정의
def encoder(input_shape):
    input_layer = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv3D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
    x = tf.keras.layers.MaxPooling3D(pool_size=2)(x)
    x = tf.keras.layers.Conv3D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling3D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    encoded = tf.keras.layers.Dense(256, activation='relu')(x)
    return tf.keras.models.Model(input_layer, encoded)

# 디코더 모델 정의
def decoder(encoded_shape, output_shape):
    input_layer = tf.keras.layers.Input(shape=encoded_shape)
    x = tf.keras.layers.Dense(256, activation='relu')(input_layer)
    x = tf.keras.layers.Reshape((8, 8, 8, 2))(x)
    x = tf.keras.layers.Conv3DTranspose(128, kernel_size=3, activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling3D(size=2)(x)
    x = tf.keras.layers.Conv3DTranspose(64, kernel_size=3, activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling3D(size=2)(x)
    decoded = tf.keras.layers.Conv3DTranspose(1, kernel_size=3, activation='sigmoid', padding='same')(x)
    return tf.keras.models.Model(input_layer, decoded)

# 인코더 모델과 디코더 모델 연결
input_shape = (32, 32, 32, 1)
encoded_shape = (256,)
encoder_model = encoder(input_shape)
decoder_model = decoder(encoded_shape, input_shape)
autoencoder_model = tf.keras.models.Model(encoder_model.input, decoder_model(encoder_model.output))

# 오토인코더 모델 컴파일
autoencoder_model.compile(optimizer='adam', loss='binary_crossentropy')

# 모델 요약 정보 출력
autoencoder_model.summary()


In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

# 데이터 불러오기
with h5py.File('./ModelNet40Vox32_airplane.h5', 'r') as f:
    data = f['data'][:]

print("Data shape:", data.shape)

# 훈련 데이터와 시험 데이터로 나누기
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

In [ ]:
# 오토인코더 모델 훈련
autoencoder_model.fit(train_data, train_data, epochs=10, batch_size=32)

In [ ]:
# 시험 데이터에 대한 재구성 결과 생성
reconstructed_data = autoencoder_model.predict(test_data)

In [ ]:
import matplotlib.pyplot as plt

# 재구성 결과 시각화
num_samples = 10  # 시각화할 샘플의 수
fig, axes = plt.subplots(nrows=num_samples, ncols=2, figsize=(10, 20))
for i in range(num_samples):
    axes[i, 0].imshow(test_data[i, ..., 0], cmap='gray')
    axes[i, 0].axis('off')
    axes[i, 0].set_title('Input')

    axes[i, 1].imshow(reconstructed_data[i, ..., 0], cmap='gray')
    axes[i, 1].axis('off')
    axes[i, 1].set_title('Reconstructed')

plt.tight_layout()
plt.show()


In [ ]:
# 재구성 오차 계산
reconstruction_error = np.mean(np.abs(test_data - reconstructed_data))

# 재구성 오차 출력
print(f"Reconstruction Error: {reconstruction_error}")